In [35]:
from utils import read_lines
from collections import defaultdict

def parse_program(s):
    return [int(x) for x in s.split(',')]

def parse_code(num):
    if num == 99:
        return num, [0]
    else:
        digits = []
        while num > 0:
            digits.append(num % 10)
            num //=10
        digits += [0] * (5-len(digits))
        opcode = digits[0]
        param_modes = digits[2:]
        return opcode, param_modes

def run(program, inputs):
    i = 0
    output = 0
    extend_memory = defaultdict(int)
    relative_base = 0
    input_pos = 0
    outputs = []
    def read_memory(addr):
        if addr < len(program):
            return program[addr]
        else:
            return extend_memory[addr]
        
    def write_memory(idx, value, mode):
        addr = program[idx]
        if mode == 2:
            addr = relative_base + addr
        if addr < len(program):
            program[addr] = value
        else:
            extend_memory[addr] = value
    
    def get_oprand(idx, mode):
        if mode == 1:
            return program[idx]
        if mode == 2:
            addr = relative_base + program[idx]
        else:
            addr = program[idx]
        return read_memory(addr)

    while i < len(program):
        opcode, param_modes = parse_code(program[i])
        if opcode in (1, 2, 5, 6, 7, 8):
            oprand1 = get_oprand(i+1, param_modes[0])
            oprand2 = get_oprand(i+2, param_modes[1])
    
            if opcode == 1:
                write_memory(i+3, oprand1 + oprand2, param_modes[2])
                i+=4
            elif opcode == 2:
                write_memory(i+3, oprand1 * oprand2, param_modes[2])
                i+=4
            elif opcode == 5:
                if oprand1 != 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 6:
                if oprand1 == 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 7:
                write_memory(i+3, 1 if oprand1 < oprand2 else 0, param_modes[2])
                i+=4
            elif opcode == 8:
                write_memory(i+3, 1 if oprand1 == oprand2 else 0, param_modes[2])
                i+=4
        elif opcode ==3 :
            input = inputs[input_pos]
            input_pos += 1
            write_memory(i+1, input, param_modes[0])
            i += 2
        elif opcode == 4:
            oprand1 = get_oprand(i+1, param_modes[0])
            output = oprand1
            i += 2
            outputs.append(output)
        elif opcode == 9:
            oprand1 = get_oprand(i+1, param_modes[0])
            relative_base += oprand1
            i+=2
        elif program[i] == 99:
            return outputs
        else:
            raise ValueError(f'illegal code {i} {program[i]}')

def print_matrix(matrix):
    for row in matrix:
        print(''.join(row))

def part1(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)
    grid = [['.'] * 50 for _ in range(50)]
    ans = 0
    for i in range(50):
        for j in range(50):
            inputs = [j, i]
            outputs = run(program.copy(), inputs)
            if outputs[0]:
                ans += 1
                grid[i][j] = '#'
    print_matrix(grid)
    return ans
    

# track the left most position of the beam for each row(as the bottom left coner), then check the top left and top right corner
def part2(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)

    x, y = 3, 4
    assert run(program.copy(), [x, y]) == [1]
    while True:
        y += 1
        while run(program.copy(), [x, y])[0] != 1:
            x += 1
        if y > 1000:
            if run(program.copy(), [x, y-99])[0] == 1 and run(program.copy(), [x+99, y-99])[0] == 1:
                return x * 10000 + y-99
    

In [34]:
part2('inputs/day19.txt')

9231141

In [14]:
part1('inputs/day19.txt')

#.................................................
..................................................
..................................................
..................................................
...#..............................................
....#.............................................
.....#............................................
......#...........................................
......##..........................................
.......##.........................................
........#.........................................
.........#........................................
.........##.......................................
..........##......................................
...........##.....................................
............##....................................
............###...................................
.............###..................................
..............###.................................
...............###.............

186